# 00 Raster import
## Description: Access previously downloaded sat data from the raster-collection script and enable manipulation of that data by correctly addressing all the contents.
The general idea is to unzip the sentinel rasters and read the contents. Converting the singleband files into a multiband image for the scripts further down the line. This is a means of reducing file handling complexity for future scripts.
### Author: Ben Ross
#### Approx Date: 24 June 2020
Project is 'Spring Monitoring'.

In [2]:
import glob
import zipfile
import fnmatch
import rasterio

In [7]:
contentsfolder = r'D:\sentinel2'
zipped_downloads = glob.glob(contentsfolder+'\Downloads\*.zip', recursive=True)
zipped_downloads

for x in zipped_downloads:
    print('Unzipping {x}'.format(x=x))
    with zipfile.ZipFile(x,"r") as zip_ref:
        zip_ref.extractall(contentsfolder+"\Downloads")

Unzipping D:\sentinel2\Downloads\S2B_MSIL2A_20200914T001109_N0214_R073_T56JKR_20200914T021322.zip


In [9]:
extracted_downloads = glob.glob(contentsfolder+'\Downloads\*.SAFE', recursive=True)
extracted_downloads

for x in extracted_downloads:
    #generate a list called y in which all the JPEG2000 files within one sat tile are listed
    y = glob.glob(x+'\GRANULE\*\IMG_DATA\*.jp2', recursive=True)
    #y = glob.glob(x+'\GRANULE\*\IMG_DATA\R20m\*.jp2', recursive=True)
    #print('There are {y} JPEG2000 files in the target folder {x}. These will be stacked using rasterio into a single {y} band image.'.format(y=len(y), x=x))
    
    #reorganise the list so that the B8A is between 8 and 9. Also omit the TCI.
    file_list = [y[0], y[1], y[2], y[3], y[4], y[5], y[6], y[7], y[12], y[8], y[9], y[10], y[11]]

    #x = x[:-5]
    #start stacking the images using rasterio
    with rasterio.open(file_list[2]) as src0:
        meta = src0.meta

        # Update meta to reflect the number of layers
        meta.update(count = len(file_list))

        # Read each layer and write it to stack
        with rasterio.open(x + '_stack.tif', 'w', **meta) as dst:
            for id, layer in enumerate(file_list, start=1):
                with rasterio.open(layer) as src1:
                    dst.write_band(id, src1.read(1))

ValueError: the array's dtype 'uint8' does not match the file's dtype 'uint16'